In [ ]:
import pandas as pd

In [ ]:
has_ans= pd.read_csv('has_answer_squad.csv')
no_ans= pd.read_csv('no_answer_squad.csv')

In [ ]:
has_ans.shape

In [ ]:
no_ans.shape

In [ ]:
has_ans= has_ans.drop(has_ans.columns[0], axis=1)
has_ans= has_ans.drop(has_ans.columns[6], axis=1)
no_ans= no_ans.drop(no_ans.columns[0], axis=1)

In [ ]:
has_ans ['question']= has_ans ['question'].str.replace('[^\w\s]','')
no_ans ['question']= no_ans ['question'].str.replace('[^\w\s]','')
has_ans ['question']= has_ans ['question'].str.lower()
no_ans ['question']= no_ans ['question'].str.lower()

In [ ]:
has_ans = has_ans[has_ans['TF'].notna()]
no_ans = no_ans[no_ans['TF'].notna()]

In [ ]:
has_ans['question_type']=''
qty=[]
keywords = ["why", "when", "how", "what", "who", "where",'which']
for index, row in has_ans.iterrows():
    answer=row['question']
    words = answer.split()
    for key in keywords:
        if key in  words:
            qty.append(key)
    row['question_type']=qty
    qty=[]


In [ ]:
has_ans.head(5000)

In [ ]:
has_ans.question_type = has_ans.question_type.apply(lambda y: '' if len(y)==0 else y)

In [ ]:
has_ans['question_type']=has_ans['question_type'].replace('', 'other')


In [ ]:
has_ans["question_type"] = has_ans["question_type"].str[0]

In [ ]:
has_ans['question_type']=has_ans['question_type'].replace('o', 'other')

In [ ]:
has_ans.question_type.value_counts()

In [ ]:
has_ans.head()

<ul>
  <li>context - provided context</li>
  <li>id - ID of question</li>
  <li>TF - whether the prediction was correct or not (True/False)</li>
  <li>pred_answer - what was the predicted answer</li>
  <li>possible_answers - acceptable answers</li>
  <li>question - question ansked</li>
</ul>

In [ ]:
no_ans['question_type']=''
keywords = ["why", "when", "how", "what", "who", "where",'which', 'whom','whose','does']
for  (i, answer) in enumerate(no_ans['question']):
    words = answer.split()
    for key in keywords:
        if key in words:
            no_ans['question_type'][i]=key
            

            

In [ ]:
no_ans['question_type']=no_ans['question_type'].replace('', 'other')

In [ ]:
no_ans.head()

<ul>
  <li>id - ID of question</li>
  <li>TF - whether the prediction was correct or not (True/False)</li>
  <li>answer - what was the predicted answer</li>
  <li>question - question ansked</li>
</ul>

In [ ]:
has_ans.TF.value_counts()

In [ ]:
no_ans.TF.value_counts()

Corresponds with the scores: 
<ul>
    <li> "exact": 86.8441000589573,</li>
    <li>"f1": 89.48714305212785, </li>
    <li>"total": 11873, </li>
    <li>"HasAns_exact": 82.08502024291498, </li>
    <li>"HasAns_f1": 87.37868580599083, </li>
    <li>"HasAns_total": 5928,</li>
    <li>"NoAns_exact": 91.58957106812447, </li>
    <li>"NoAns_f1": 91.58957106812447, </li>
    <li>"NoAns_total": 5945, </li>
    <li>"best_exact": 86.86936747241641,</li>
    <li>"best_exact_thresh": -2.737598419189453, </li>
    <li>"best_f1": 89.51241046558647, </li>
    <li>"best_f1_thresh": -2.737598419189453</li>
 </ul>

In [ ]:
no_ans = no_ans.rename({'answer': 'pred_answer'}, axis='columns')


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
def plot_histogram(data, label='',datalabel=''):
    plt.hist(data, bins=100, normed=1, alpha=1.0)
    plt.xlabel('Lengths')
    plt.ylabel('Probability')
    plt.grid(True)
    plt.title((datalabel+' '+label+' length').capitalize())
    plt.savefig('graphs/'+datalabel+'_length_'+label+'.png')
    plt.show()
    plt.clf()

In [ ]:
plot_histogram([(len(str(a))) for a in has_ans['pred_answer']], 'answer','has_ans')
plot_histogram([q for q in has_ans['question'].str.len()], 'question','has_ans')
plot_histogram([c for c in has_ans['context'].str.len()], 'context','has_ans')

In [ ]:
has_ans['ans_length']  = has_ans['pred_answer'].str.len()
has_ans['con_length']  = has_ans['context'].str.len()
has_ans['que_length']  = has_ans['question'].str.len()

In [ ]:
def question_questiontype(dataset,label,x,datalabel):
    answers= []
    keywords = ["why", "when", "how", "what", "who", "where",'which','other']
    for key in keywords:
        answers.append([len(answer) for (i, answer) in enumerate(dataset) if key in has_ans.iloc[i]['question_type'].lower()])
    bins = np.linspace(0, x, 15)
    plt.hist(answers, bins, normed=True, label=keywords)
    plt.xlabel('Lengths')
    plt.ylabel('Probability')
    plt.grid(True)
    plt.legend()
    plt.title((datalabel+' '+label+' questiontype').capitalize())
    plt.savefig('graphs/'+datalabel+'_questiontype_'+label+'.png')
    plt.show()
    plt.clf()

In [ ]:
question_questiontype(has_ans['pred_answer'],'answer',100,'has_ans' )
question_questiontype(has_ans['context'],'context',3000,'has_ans')
question_questiontype(has_ans['question'],'question',150,'has_ans')

In [ ]:
has_ans.question_type.isnull().sum()

In [ ]:
incorrect=has_ans[has_ans['TF'] == False]
correct=has_ans[has_ans['TF']==True]

# Only incorrect answers

In [ ]:
plot_histogram([a for a in incorrect['pred_answer'].str.len()], 'answer','has_ans_incorrect')
plot_histogram([q for q in incorrect['question'].str.len()], 'question','has_ans_incorrect')
plot_histogram([c for c in incorrect['context'].str.len()], 'context','has_ans_incorrect')

In [ ]:
question_questiontype(incorrect['pred_answer'],'answer',100,'has_ans_incorrect' )
question_questiontype(incorrect['context'],'context',3000,'has_ans_incorrect')
question_questiontype(incorrect['question'],'question',150,'has_ans_incorrect')

# Only correct answers


In [ ]:
plot_histogram([a for a in correct['pred_answer'].str.len()], 'answer','has_ans_correct')
plot_histogram([q for q in correct['question'].str.len()], 'question','has_ans_correct')
plot_histogram([c for c in correct['context'].str.len()], 'context','has_ans_correct')

In [ ]:
question_questiontype(correct['pred_answer'],'answer',100,'has_ans_correct' )
question_questiontype(correct['context'],'context',3000,'has_ans_correct')
question_questiontype(correct['question'],'question',150,'has_ans_correct')

In [ ]:
sns.histplot(data=has_ans, x="ans_length",bins=100, hue='TF',palette=["C0", "C1"])
plt.show()

In [ ]:
correct=has_ans[has_ans['TF']==True]

## Pie charts


In [ ]:

def piechart (dataset,labels,name):
    values=dataset.value_counts()
    labels=dataset.value_counts().index.values
    plt.figure(figsize=(5,5))
    plt.pie(values,labels=labels,autopct='%1.1f%%',shadow=True)
    plt.legend(labels)
    plt.axis('equal')
    plt.tight_layout()
    plt.title((name+' question type distribution').capitalize())
    plt.savefig('graphs/'+name+'_questiontype_pie.png')
    plt.show()
    print(dataset.value_counts())



In [ ]:
labels=''
piechart(has_ans['question_type'],labels,'has_ans')
piechart(incorrect['question_type'],labels,'has_ans_incorrect')
piechart(correct['question_type'],labels,'has_ans_correct')

In [ ]:
def pie_in(labels,dataset,name,feature):
    plt.figure(figsize=(4,4))
    values=dataset.value_counts().values
    labels=dataset.value_counts().index.values
    plt.pie(values,labels=labels,autopct='%1.1f%%',shadow=True)
    plt.legend(labels)
    plt.axis('equal')
    plt.tight_layout()
    plt.title((name+' '+feature+' attribute distribution').capitalize())
    plt.savefig('graphs/'+name+'_'+feature+'_pie.png')
    plt.show()

# Unanswerable part

In [ ]:
plot_histogram([(len(str(a))) for a in no_ans['pred_answer']], 'answer','no_ans')
plot_histogram([q for q in no_ans['question'].str.len()], 'question','no_ans')
plot_histogram([c for c in no_ans['context'].str.len()], 'context','no_ans')

In [ ]:
no_ans['ans_length']  = no_ans['pred_answer'].str.len()
no_ans['con_length']  = no_ans['context'].str.len()
no_ans['que_length']  = no_ans['question'].str.len()

In [ ]:
def question_questiontype(dataset,label,x,datalabel):
    answers= []
    keywords = ["why", "when", "how", "what", "who", "where",'which','other']
    for key in keywords:
        answers.append([len(answer) for (i, answer) in enumerate(dataset) if key in no_ans.iloc[i]['question_type'].lower()])
    bins = np.linspace(0, x, 15)
    plt.hist(answers, bins, normed=True, label=keywords)
    plt.xlabel('Lengths')
    plt.ylabel('Probability')
    plt.grid(True)
    plt.legend()
    plt.title((label+' questiontype').capitalize())
    plt.savefig('graphs/'+datalabel+'_questiontype_'+label+'.png')
    plt.show()
    plt.clf()

In [ ]:
question_questiontype(no_ans['pred_answer'],'answer',100,'no_ans' )
question_questiontype(no_ans['context'],'context',3000,'no_ans')
question_questiontype(no_ans['question'],'question',150,'no_ans')

In [ ]:
incorrect=no_ans[no_ans['TF'] == False]
correct=no_ans[no_ans['TF']==True]

## Only incorrect answers

In [ ]:
plot_histogram([a for a in incorrect['pred_answer'].str.len()], 'answer','no_ans_incorrect')
plot_histogram([q for q in incorrect['question'].str.len()], 'question','no_ans_incorrect')
plot_histogram([c for c in incorrect['context'].str.len()], 'context','no_ans_incorrect')

In [ ]:
question_questiontype(incorrect['pred_answer'],'answer',100,'no_ans_incorrect' )
question_questiontype(incorrect['context'],'context',3000,'no_ans_incorrect')
question_questiontype(incorrect['question'],'question',150,'no_ans_incorrect')

## Only correct answers

In [ ]:
plot_histogram([a for a in correct['pred_answer'].str.len()], 'answer','no_ans_correct')
plot_histogram([q for q in correct['question'].str.len()], 'question','no_ans_correct')
plot_histogram([c for c in correct['context'].str.len()], 'context','no_ans_correct')

In [ ]:
question_questiontype(correct['pred_answer'],'answer',100,'no_ans_correct' )
question_questiontype(correct['context'],'context',3000,'no_ans_correct')
question_questiontype(correct['question'],'question',150,'no_ans_correct')

## Pies

In [ ]:

piechart(no_ans['question_type'],labels,'no_ans')
piechart(incorrect['question_type'],labels,'no_ans_incorrect')
piechart(correct['question_type'],labels,'no_ans_correct')

# Whole set

In [ ]:
ans=pd.merge(has_ans,no_ans,how='outer')

In [ ]:
plot_histogram([(len(str(a))) for a in ans['pred_answer']], 'answer','ans')
plot_histogram([q for q in ans['question'].str.len()], 'question','ans')
plot_histogram([c for c in ans['context'].str.len()], 'context','ans')

In [ ]:
ans['ans_length']  = ans['pred_answer'].str.len()
ans['con_length']  = ans['context'].str.len()
ans['que_length']  = ans['question'].str.len()

In [ ]:
def question_questiontype(dataset,label,x,datalabel):
    answers= []
    keywords = ["why", "when", "how", "what", "who", "where",'which','other']
    for key in keywords:
        answers.append([len(answer) for (i, answer) in enumerate(dataset) if key in ans.iloc[i]['question_type'].lower()])
    bins = np.linspace(0, x, 15)
    plt.hist(answers, bins, normed=True, label=keywords)
    plt.xlabel('Lengths')
    plt.ylabel('Probability')
    plt.grid(True)
    plt.legend()
    plt.title((label+' questiontype').capitalize())
    plt.savefig('graphs/'+datalabel+'_questiontype_'+label+'.png')
    plt.show()
    plt.clf()

In [ ]:
question_questiontype(ans['pred_answer'],'answer',100,'ans' )
question_questiontype(ans['context'],'context',3000,'ans')
question_questiontype(ans['question'],'question',150,'ans')

In [ ]:
incorrect=ans[ans['TF'] == False]
correct=ans[ans['TF']==True]

## Only incorrect answers

In [ ]:
plot_histogram([a for a in incorrect['pred_answer'].str.len()], 'answer','ans_incorrect')
plot_histogram([q for q in incorrect['question'].str.len()], 'question','ans_incorrect')
plot_histogram([c for c in incorrect['context'].str.len()], 'context','ans_incorrect')

In [ ]:
question_questiontype(incorrect['pred_answer'],'answer',100,'ans_incorrect' )
question_questiontype(incorrect['context'],'context',3000,'ans_incorrect')
question_questiontype(incorrect['question'],'question',150,'ans_incorrect')

## Only correct answers

In [ ]:
plot_histogram([a for a in correct['pred_answer'].str.len()], 'answer','ans_correct')
plot_histogram([q for q in correct['question'].str.len()], 'question','ans_correct')
plot_histogram([c for c in correct['context'].str.len()], 'context','ans_correct')

In [ ]:
question_questiontype(correct['pred_answer'],'answer',100,'ans_correct' )
question_questiontype(correct['context'],'context',3000,'ans_correct')
question_questiontype(correct['question'],'question',150,'ans_correct')

## Pies

In [ ]:
piechart(ans['question_type'],labels,'SQUAD Dataset')
piechart(incorrect['question_type'],labels,'SQUAD - Incorrect predictions')
piechart(correct['question_type'],labels,'SQUAD - Correct predictions')

# question type analysis

In [ ]:
# crosstab
df=ans.sort_values('TF')
pal = ["red","royalblue" ]
ax= pd.crosstab(df['question_type'],df['TF'] ).apply(lambda r: r/r.sum()*100, axis=1)
ax=ax.sort_values([1], ascending=True)
ax_1 = ax.plot.bar(figsize=(10,6),stacked=True, rot=0,color=pal, sort_columns=True)
display(ax)

plt.legend(loc='upper center', bbox_to_anchor=(1.0, 1.0), title="Correct Prediction")

plt.xlabel('Question type')
plt.ylabel('Percent Distribution')

for rec in ax_1.patches:
    height = rec.get_height()
    ax_1.text(rec.get_x() + rec.get_width() / 2, 
              rec.get_y() + height / 2,
              "{:.0f}%".format(height),
              ha='center', 
              va='bottom')
plt.savefig('graphs/SQUAD_questiontype_distribution.png')
plt.show()

In [ ]:
ax.sort_values([1], ascending=True)

# How analysis

In [ ]:

how_analysis=pd.merge(has_ans,no_ans,how='outer')


In [ ]:
how_analysis=how_analysis.reset_index(drop=True)
how_analysis['question']=how_analysis['question'].str.lower()


In [ ]:
how_analysis.head()


In [ ]:
how_analysis['question_type']=''
qty=[]
keywords = ['how','far','long','many', 'much','old', 'come']
for index, row in how_analysis.iterrows():

    answer=row['question']
    words = answer.split()
    for key in keywords:
        if key in  words:
            qty.append(key)
    how_analysis['question_type'][index]=qty
    qty=[]



In [ ]:
how_analysis=how_analysis[how_analysis.question_type.apply(lambda x: len(x) > 1)]

In [ ]:
how_analysis.question_type.value_counts()

In [ ]:
how_analysis=how_analysis[how_analysis['question_type']!='']

In [ ]:
how_analysis['question_type'] = how_analysis['question_type'].str[1]

In [ ]:
gh=how_analysis[how_analysis['question_type']=='many']
gh.sample(6)

In [ ]:
# crosstab
df=how_analysis
pal = ["red","royalblue" ]
ax= pd.crosstab(df['question_type'],df['TF'] ).apply(lambda r: r/r.sum()*100, axis=1)
ax=ax.sort_values([1], ascending=True)
ax_1 = ax.plot.bar(figsize=(10,6),stacked=True, rot=0,color=pal, sort_columns=True)
display(ax)

plt.legend(loc='upper center', bbox_to_anchor=(1.0, 1.0), title="Correct Prediction")

plt.xlabel('Question type')
plt.ylabel('Percent Distribution')

for rec in ax_1.patches:
    height = rec.get_height()
    ax_1.text(rec.get_x() + rec.get_width() / 2, 
              rec.get_y() + height / 2,
              "{:.0f}%".format(height),
              ha='center', 
              va='bottom')
plt.savefig('graphs/how_questiontype_distribution.png')
plt.show()

# ERROR ANALYSIS

# incorrectly preddicted NoAns where answer existed

In [ ]:
incorrect.loc[~incorrect.pred_answer.isin(['-']), 'pred_answer'] = 'Other'
incorrect.pred_answer.value_counts()

In [ ]:
noanswrg=incorrect[(incorrect['pred_answer'] == '-')]  


In [ ]:

piechart(noanswrg['question_type'],labels,'')

# question is unasnwerable but the answer was predicted

In [ ]:
incorrect.possible_answers.isnull().sum()

In [ ]:
unans=incorrect[incorrect.possible_answers.isnull()==True]  


In [ ]:
piechart(unans['question_type'],labels,'')

In [ ]:
#restoring to original
incorrect=ans[ans['TF'] == False]


In [ ]:
#for copying examples
incorrect[~(incorrect['possible_answers'] == 'NaN')]  

In [ ]:
incorrect.tail(50)

In [ ]:
inc=ans[ans['TF'] == False]


In [ ]:
#dropping the previous two groups
inc=inc[~(inc['pred_answer'] == '-')]  
inc = inc[inc['possible_answers'].notna()]

In [ ]:
#testing if everything is gone
inc.possible_answers.isnull().sum()

In [ ]:
inc[(inc['pred_answer'] == '-')]  

# preparation for the upcoming checks whether the predicted answer contains the original answer

In [ ]:
#exploding possible answers since there may be more than one
inc['possible_answers'] = inc['possible_answers'].str.replace('[', '').astype(str)
inc['possible_answers'] = inc['possible_answers'].str.replace(']', '').astype(str)

In [ ]:
inc['possible']=inc['possible_answers']

In [ ]:
inc.head()

In [ ]:
inc= inc.assign(possible=inc['possible'].str.split('|')).explode('possible')

In [ ]:
#if the original answer is present in the predicted anser
#(that means the model picked longer sequence of words than the original answer.
#the information conveyed is still correct.)
inc['answer_text_in_prediction'] = inc.apply(lambda x: x.possible in x.pred_answer, axis=1)
inc['answer_prediction_in_answer_text'] = inc.apply(lambda x: x.pred_answer in x.possible_answers, axis=1)

In [ ]:
inc.head()

In [ ]:
#caring only about the ones that are longer
inc['poss_length']  = inc['possible'].str.len()
inc['difference'] = inc.apply(lambda x: x.poss_length < x.ans_length, axis=1)
inc['difference2'] = inc.apply(lambda x: x.poss_length > x.ans_length, axis=1)

In [ ]:
inc.head()

# predicted answer is longer than the original but still conveys correct information

In [ ]:
#grouping back after the necessary checks
inc=inc.groupby('id').agg(lambda x: x.tolist())

In [ ]:
#creating attribute to check whether if there are such instances 
inc['kkt'] = inc.apply(lambda x: any(x.answer_text_in_prediction) &  any(x.difference), axis=1)

In [ ]:
inc.kkt.value_counts()

In [ ]:
pd.set_option('display.max_colwidth', None)

# checking whether the predicted answer is shorter than the original answer but still conveys same information

In [ ]:
inc['kkt2'] = inc.apply(lambda x: any(x.answer_prediction_in_answer_text) &  any(x.difference2), axis=1)

In [ ]:
inc.kkt2.value_counts()

# since there are several possible answers for one question, its important to drop instances where some plausible answer can fulfil the condition of being longer than predicted answer and the other plausible answer for the same question fulfilling condition of beingh shorter

In [ ]:
example=inc[(inc['kkt'] == True)& (inc['kkt2'] == True)]  
example.sample()

In [ ]:
example.kkt.value_counts()

In [ ]:
inc=inc[((inc['kkt'] & inc['kkt2']) == False)]  


In [ ]:
merge=pd.merge(incorrect,inc,how='right', on='id')

In [ ]:
merge.head()

In [ ]:
longer=merge[(merge['kkt'] == True)]
longer.kkt.value_counts()

In [ ]:
shorter=merge[(merge['kkt2'] == True)]  
shorter.kkt2.value_counts()

In [ ]:
piechart(shorter['question_type_x'],labels,'')

In [ ]:
piechart(longer['question_type_x'],labels,'')

In [ ]:
longer.sample()

In [ ]:
shorter.sample()

# just wrong answers

In [ ]:
wrong=merge[(merge['kkt2'] == False)&(merge['kkt'] == False)]  

In [ ]:
wrong.shape

In [ ]:
wrong.sample()

In [ ]:
piechart(wrong['question_type_x'],labels,'')

# analysis of char-count

In [ ]:
answ=ans
def analyse_lowest(part, label):
    se=100000
    for q in part.str.len():
        if se>q:
            se=q
            
    print('The length of ',label,'starts at ',se)
def analyse_highest(part, label):
    se=0
    for q in part.str.len():
        if se<q:
            se=q
            
    print('The length of ',label,'ends at ',se)
answ = answ[answ['possible_answers'].notna()]
#exploding possible answers
answ['possible_answers'] = answ['possible_answers'].str.replace('[', '').astype(str)
answ['possible_answers'] = answ['possible_answers'].str.replace(']', '').astype(str)
answ['possible']=answ['possible_answers']    
answ= answ.assign(possible=answ['possible'].str.split('|')).explode('possible')

In [ ]:
analyse_lowest(answ['possible'],'original answer')
analyse_lowest(answ['context'],'context')
analyse_lowest(answ['question'],'question')
analyse_lowest(answ['pred_answer'],'pred_answer')
analyse_highest(answ['possible'],'original answer')
analyse_highest(answ['context'],'context')
analyse_highest(answ['question'],'question')
analyse_highest(answ['pred_answer'],'pred_answer')

# questiontype samples

In [ ]:
has_ans[has_ans['question_type']=='other']



In [ ]:

wh=no_ans[no_ans['question_type']=='other']
wh.sample()

# distribution of all questiontype pairs

In [ ]:

ans['question_type']=''
qty=[]
keywords = ["why", "when", "how", "what", "who", "where",'which', 'whom','whose','does']
for index, row in ans.iterrows():

    answer=row['question']
    words = answer.split()
    for key in keywords:
        if key in  words:
            qty.append(key)
    ans['question_type'][index]=qty
    qty=[]
ans.question_type.value_counts()
